In [36]:
import pyodbc
import sqlalchemy
import pandas as pd
import time

In [12]:
additions_list = pd.read_excel(r'T:\AArete Client Work\Blue Cross Blue Shield CA (Promise Health Plan)\Restricted\3_CoHC_Support\Mike S\DOFR Review\DOFR_Matrix_Additions_20210910.xlsx', sheet_name='Final List Additions')
top_spend_ids = pd.read_excel(r'T:\AArete Client Work\Blue Cross Blue Shield CA (Promise Health Plan)\Restricted\3_CoHC_Support\Mike S\DOFR Review\DOFR_Matrix_Additions_20210910.xlsx', sheet_name='Spend by IPA')

additions_list.info()
print()
top_spend_ids.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7550 entries, 0 to 7549
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   DOFR ID    7550 non-null   int64 
 1   IPA ID     7550 non-null   object
 2   Serv_Cat   7550 non-null   object
 3   In Matrix  7550 non-null   object
dtypes: int64(1), object(3)
memory usage: 236.1+ KB

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46 entries, 0 to 45
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   DOFR ID       46 non-null     int64  
 1   IPA ID        46 non-null     object 
 2   Annual Spend  46 non-null     float64
dtypes: float64(1), int64(1), object(1)
memory usage: 1.2+ KB


In [15]:
top_spend_ids = top_spend_ids[top_spend_ids['IPA ID'].isin(additions_list['IPA ID'])]
top_spend_ids.shape

(40, 3)

In [37]:
def printProgressBar (iteration, total, prefix = 'Progress', suffix = 'Complete', decimals = 1, length = 100, fill = '█', printEnd = "\r"):
    """
    Call in a loop to create terminal progress bar
    @params:
        iteration   - Required  : current iteration (Int)
        total       - Required  : total iterations (Int)
        prefix      - Optional  : prefix string (Str)
        suffix      - Optional  : suffix string (Str)
        decimals    - Optional  : positive number of decimals in percent complete (Int)
        length      - Optional  : character length of bar (Int)
        fill        - Optional  : bar fill character (Str)
        printEnd    - Optional  : end character (e.g. "\r", "\r\n") (Str)
    """
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    print(f'\r{prefix} |{bar}| {percent}% {suffix}', end = printEnd)
    # Print New Line on Complete
    if iteration == total: 
        print()

In [42]:
def mod_DOFR_mat(ipa_id, dofr_id):
    conn = pyodbc.connect('Driver={SQL Server};'
                          'Server=AARSVRSQL;'
                          'Database=BSCP_DW;'
                          'Trusted_Connection=yes;')

    serv_cats = additions_list.loc[additions_list['IPA ID'] == ipa_id, 'Serv_Cat']
    
    cursor = conn.cursor()
    cursor.execute('DROP TABLE IF EXISTS BSCP_DW.TMP.DOFTMAtrixAdd')
    cursor.execute('SELECT * INTO BSCP_DW.TMP.DOFRMatrixAdd FROM BSCP_DW.TMP.CompleteDOFRMatrix')
    cursor.execute(f'DELETE FROM BSCP_DW.TMP.DOFRMatrixAdd WHERE Serv_Cat IN {tuple(serv_cats)}')
    
    for i, serv_cat in enumerate(serv_cats):
        dofr_mat = pd.read_sql(f"SELECT * FROM [BSCP_DW].[TMP].[CompleteDOFRMatrix] WHERE Serv_Cat = '{serv_cat}'", con=conn)

        dofr_mat['DOFR_ID'] = dofr_mat['DOFR_ID'].map(lambda x: [x])
        dofr_mat['IPA_ID'] = dofr_mat['IPA_ID'].map(lambda x: [x])

        dofr_mat['DOFR_ID'] = dofr_mat['DOFR_ID'].map(lambda x: x + [dofr_id])
        dofr_mat['IPA_ID'] = dofr_mat['IPA_ID'].map(lambda x: x + [ipa_id])
        
        dofr_mat.to_sql('BSCP_DW.TMP.DOFRMatrixAdd', con=conn, if_exists='append')
        dofr_mat_id = pd.concat([dofr_mat_id, dofr_mat]).reset_index(drop=True)

        time.sleep(0.1)
        printProgressBar(i + 1, serv_cats.shape[0])
        

In [43]:
ipa_id = top_spend_ids['IPA ID'].iloc[0]
dofr_id = top_spend_ids['DOFR ID'].iloc[0]

mod_DOFR_mat(ipa_id, dofr_id)

DatabaseError: Execution failed on sql 'SELECT name FROM sqlite_master WHERE type='table' AND name=?;': ('42S02', "[42S02] [Microsoft][ODBC SQL Server Driver][SQL Server]Invalid object name 'sqlite_master'. (208) (SQLExecDirectW); [42S02] [Microsoft][ODBC SQL Server Driver][SQL Server]Statement(s) could not be prepared. (8180)")